In [21]:
from tensorflow.keras.models import Sequential,Model,save_model
from tensorflow.keras.layers import Dense, Activation, Dropout, RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import add

In [2]:
from _pickle import dump,load

In [4]:
x1_train = load(open('x1_train.pkl','rb'))
x2_train = load(open('x2_train.pkl','rb'))
y_train = load(open('y_train.pkl','rb'))
embedding = load(open('embedding.pkl','rb'))
tokenizer = load(open('tokenizer.pkl','rb'))
index_to_word = load(open('index_to_word.pkl','rb'))

In [30]:
maxlen_x = len(x1_train[0])
maxlen_y = len(x2_train[0])
vocab_size = embedding.shape[0]

print(maxlen_x,maxlen_y,vocab_size)

1247 9 6222


In [5]:
activation_rnn_size = 40 
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False
seed=42
p_W, p_U, p_dense, weight_decay = 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [6]:
regularizer = l2(0)

In [35]:

input1 = Input(shape=(maxlen_x,),name='input1')
text_em = Embedding(vocab_size, 200,weights=[embedding],embeddings_regularizer=regularizer,\
                    mask_zero=True,name='text_em')(input1)
text_drop=Dropout(.5,name='text_drop')(text_em)
#text_lstm1= LSTM(600,activation='relu',return_sequences=True,dropout=.5,kernel_regularizer=regularizer ,name='text_lstm')(text_drop)
#text_lstm2= LSTM(30,activation='relu',return_sequences=True,dropout=.5,kernel_regularizer=regularizer ,name='text_lstm1')(text_drop)
text_lstm3= LSTM(12,activation='relu',dropout=.5,kernel_regularizer=regularizer ,name='text_lstm2')(text_drop)


#encoder13 = RepeatVector(sum_txt_length)(encoder2)

input2 = Input(shape=(maxlen_y,),name='input2')
head_em = Embedding(vocab_size, 200,weights=[embedding],embeddings_regularizer=regularizer,\
                    mask_zero=True,name='head_em')(input2)
head_drop=Dropout(.5,name='head_drop')(head_em)
head_lstm1 = LSTM(12,dropout=.5,activation='relu',kernel_regularizer=regularizer ,name='head_lstm1')(head_drop)
#head_lstm2= LSTM(30,activation='relu',dropout=.5,kernel_regularizer=regularizer ,name='head_lstm2')(head_lstm1)
#encoder23 = RepeatVector(sum_txt_length)(encoder2)

decoder_add = add([text_lstm3,head_lstm1],name='decoder_add')
decoder_lstm= Dense(256,activation='relu',name='decoder_dence1')(decoder_add)
output = Dense(vocab_size, activation='softmax',name='decoder_dence2')(decoder_lstm)
# tie it together
model = Model(inputs=[input1,input2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 1247)]       0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 9)]          0                                            
__________________________________________________________________________________________________
text_em (Embedding)             (None, 1247, 200)    1244400     input1[0][0]                     
__________________________________________________________________________________________________
head_em (Embedding)             (None, 9, 200)       1244400     input2[0][0]                     
____________________________________________________________________________________________

In [37]:
model.fit([x1_train,x2_train],y_train,verbose=2,batch_size=200,epochs=20)

Train on 630 samples
Epoch 1/20
630/630 - 3s - loss: 5.1771
Epoch 2/20
630/630 - 3s - loss: 5.1658
Epoch 3/20
630/630 - 3s - loss: 5.1871
Epoch 4/20
630/630 - 3s - loss: 5.1440
Epoch 5/20
630/630 - 3s - loss: 5.1435
Epoch 6/20
630/630 - 3s - loss: 5.1281
Epoch 7/20
630/630 - 4s - loss: 5.1065
Epoch 8/20
630/630 - 4s - loss: 5.1090
Epoch 9/20
630/630 - 4s - loss: 5.0847
Epoch 10/20
630/630 - 5s - loss: 5.0864
Epoch 11/20
630/630 - 4s - loss: 5.0593
Epoch 12/20
630/630 - 4s - loss: 5.0481
Epoch 13/20
630/630 - 4s - loss: 5.0243
Epoch 14/20
630/630 - 4s - loss: 5.0228
Epoch 15/20
630/630 - 4s - loss: 4.9909
Epoch 16/20
630/630 - 4s - loss: 4.9937
Epoch 17/20
630/630 - 4s - loss: 4.9827
Epoch 18/20
630/630 - 4s - loss: 4.9451
Epoch 19/20
630/630 - 3s - loss: 4.9394
Epoch 20/20
630/630 - 3s - loss: 4.9027


In [45]:
import os
import codecs
from pandas import DataFrame

In [41]:
os.getcwd()

'/home/aninda/protoTypeProject/nlp/Auto Headline Generator Using LSTM'

In [46]:
test = DataFrame(columns=['Text']) 
cnt=0
for file in os.listdir('./testData'):
    test=test.append({'Text':codecs.open('./testData/'+file,encoding='utf8',errors='replace').read()},ignore_index=True)
    cnt+=1
    print(file)

063.txt
119.txt
043.txt
051.txt
001.txt
183.txt
323.txt
188.txt
057.txt
026.txt
111.txt
037.txt
028.txt
060.txt
207.txt


In [47]:
test['head_lines']=test['Text'].apply(lambda x: 'startseq '+x.splitlines()[0]+' endseq')
test['Text']=test['Text'].apply(lambda x: 'startseq '+' '.join(list(filter(bool,x.splitlines()))[1:])+' endseq')

In [59]:
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import argmax

In [55]:
test_sequence_text=tokenizer.texts_to_sequences(test["Text"].tolist())


In [67]:
test1=pad_sequences([test_sequence_text[10]],maxlen=1247)

In [63]:
def pred_description(model,tokenizer,doc,mx_len=9,):
    in_text='startseq'

    for i in range(mx_len):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=mx_len)
        pred = model.predict([doc,sequence], verbose=0)
        pred = argmax(pred)
        print(pred)
        pred_word = index_to_word[pred]
        in_text += ' '+pred_word
        if pred_word == 'endseq' or pred_word is None:
            break
    in_text=' '.join(in_text.split()[1:-1])
    return in_text

In [68]:
a=pred_description(model,tokenizer,test1)

2
2
9
19


In [69]:
a

'to to for'